# <center>Crash course 1: Vectors and matrices in numpy and scipy</center>
### <center>Alfred Galichon (NYU & ScPo) and Clément Montes (ScPo)</center>
## <center>'math+econ+code' masterclass on optimal transport and economic applications</center>
#### <center>With python code examples</center>
© 2018-2022 by Alfred Galichon. Past and present support from NSF grant DMS-1716489, ERC grant CoG-866274 are acknowledged, as well as inputs from contributors listed [here](http://www.math-econ-code.org/theteam).

**If you reuse material from this masterclass, please cite as:**<br>
Alfred Galichon, 'math+econ+code' masterclass on optimal transport and economic applications, January 2022. https://github.com/math-econ-code/mec_optim

# Introducing NumPy

* Unlike R or Matlab, Python has no built-in matrix algebra interface. Fortunately, the NumPy library provides powerful matrix capabilities, on par with R or Matlab. Here is a quick introduction to vectorization, operations on vectors and matrices, higher-dimensional arrays, Kronecker products and sparse matrices, etc. in NumPy.

* This is *not* a tutorial on Python itself. They are plenty good ones available on the web.

* First, we load numpy (with its widely used alias):

In [1]:
import numpy as np

In NumPy, an `array` is built from a lists as follows:

In [2]:
u = np.array([1,2,3])
print(u)
v = np.array([3,2,5])
print(v)

[1 2 3]
[3 2 5]


One can then add arrays as:

In [5]:
print(np.array([1,2,3])+np.array([3,2,5]))

[4 4 8]


Note the difference between the + operator when applied to numpy arrays vs. when applied to lists:

In [8]:
[1,2,3]+[3,2,5]

[1, 2, 3, 3, 2, 5]

In [9]:
["Samya"]+["25 octobre"]+["1997"]

['Samya', '25 octobre', '1997']

In the latter case, it returns list concatenation.

To input matrices in NumPy, one simply inputs a list of rows, which are themselves represented as lists.

In [10]:
A = np.array([[11,12],[21,22],[31,32]])
A

array([[11, 12],
       [21, 22],
       [31, 32]])

The `shape` attribute of an array indicated the dimension of that array.

In [11]:
A.shape

(3, 2)

## Vectorization and memory order

* Matrices in all mathematical softwares are represented in a *vectorized* way as a sequence of numbers in the computers memory. This representation can involve either stacking the lines, or stacking the columns.

* Different programming languages can use either of the two stacking conventions:
    + Stacking the lines (Row-major order) is used by `C`, and is the default convention for Python (NumPy). A matrix $M$ is represented by varying the last index first, i.e. a $2\times2$ matrix will be represented as $vec_C\left(M\right) = \left(M_{11}, M_{12}, M_{21}, M_{22}\right).$ 
    + Stacking the columns (Column-major order) is used by `Fortran`, `Matlab`, `R`, and most underlying core linear algebra libraries (like BLAS). A 2x2x2 3-dimensional array $A$ will be represented by varying the first index first, then the second, i.e. $vec_C\left(A\right) = \left( A_{111}, A_{211}, A_{121}, A_{221}, A_{112}, A_{212}, A_{122}, A_{222} \right)$. 

### Remarque : Je ne comprends pas l'intérêt d'avoir une convention (pour R, Matlab, Fortran) qui ne soit pas celle des mathématiques

The command `flatten()` provides the vectorized representation of a matrix.

In [13]:
A.flatten()

array([11, 12, 21, 22, 31, 32])

In [14]:
(A + A).flatten()

array([22, 24, 42, 44, 62, 64])

Remember, NumPy represents matrices by **varying the last index first**.

In order to reshape the matrix `a`, one modifies its `shape` attribute. The following reshapes the matrix `a` into a row vector. 

In [15]:
A.shape = 1,6
A

array([[11, 12, 21, 22, 31, 32]])

The previous output evidences the fact that Python uses the row-major order: rows are stacked one after the other. 
To reshape the vector into a column vector, do:

In [16]:
A.shape = 6,1
A

array([[11],
       [12],
       [21],
       [22],
       [31],
       [32]])

In [26]:
A.shape = -1,3,2
A

array([[[11, 12],
        [21, 22],
        [31, 32]]])

### -1 : Pas mal pour ne pas avoir à me rappeler du nombre de lignes dans un vector colonnes compliqué par exemple

Equivalently, one could have set `A.shape=6,-1`, where Python would replace `-1` by the integer needed for the formula to make sense (in this case, `1`). 
Another way to reshape is to use the method `reshape,` which returns a duplicate of the object with the requested shape.

In [ ]:
A1=np.array(range(6))
A2 = A1.reshape(3,2)
print("A1=\n", A1)
print("A2=\n",A2)

Note that `NumPy` also supports the column-major order, but you have to specifically ask for it, by passing the optional argument `order='F'`, where 'F' stands for `Fortran`.

In [ ]:
A3 = np.array(range(6)).reshape(3,2, order='F')
A3

# Multiplication 

### Multiplication of arrays

There are several ways to multiply two arrays using NumPy. The most commonly used is the following.

In [30]:
A = np.ones((2,2))
B = 3*np.eye(2)
A@B #@ is left associative. If you have A@B@C, it will compute (A@B)@C

array([[3., 3.],
       [3., 3.]])

Note that `np.matmul(A,B)` would give the same result as well, but it is more difficult to read `np.matmul(A,np.matmul(B,C))` than `A@B@C`.

### Multiplication by a scalar

In [28]:
4*np.eye(2)

array([[4., 0.],
       [0., 4.]])

The above assignation of B corresponds to the multiplication by a scalar. It is the simplest broadcasting allowed by numpy (which makes this library more powerful than just using lists -it is also much quicker-). More on broadcasting will arrive later in that Notebook.

Je suppose que 4 est broadcasted dans 4*np.eye(2), de sortes que multiplier par 4 revient à faire le produit matriciel par 4*np.eye(2) càd une homothétie

## Kronecker product

A very important identity is
\begin{align*}
vec_C\left(AXB\right) = \left(  A\otimes B^\top\right)  vec_C\left(X\right),
\end{align*}
where $vec_C$ is the vectorization under the C (row-major) order, and where the Kronecker product $\otimes$ is defined as follows for 2x2 matrices (with obvious generalization):

\begin{align*}
A\otimes B=
\begin{pmatrix}
a_{11}B & a_{12}B\\
a_{21}B & a_{22}B
\end{pmatrix}.
\end{align*}



In [ ]:
A = np.eye(2)

AXB = np.kron(A, B)
print("A=",A)
print("B=",B)
print("AXB=",AXB)

# Vérifier l'égalité en question

In [34]:
AAB = A@A@B
LHS = AAB.flatten()
RHS = np.kron(A,B.T)@A.flatten()

In [36]:
print(LHS,RHS)
print(np.all(LHS == RHS))

[6. 6. 6. 6.] [6. 6. 6. 6.]
True


## Type broadcasting in NumPy


The term broadcasting describes how NumPy treats arrays with different shapes during arithmetic operations. 

Subject to certain constraints, the smaller array is “broadcasted” across the larger array so that they have compatible shapes. Broadcasting provides a means of vectorizing array operations.

In [37]:
A = 10*np.array([[1],[2],[3]]) #Simplest broadcasting
B =  np.array([1,2])
print('A=\n',A)
print('B=\n',B)
print('A+B=\n',A+B)

A=
 [[10]
 [20]
 [30]]
B=
 [1 2]
A+B=
 [[11 12]
 [21 22]
 [31 32]]


In [45]:
print(A.shape)
print(B.shape)
print((A+B).shape)

(3, 1)
(2,)
(3, 2)


# Donc c'est plus compliqué l'opération +, pas mal de broadcasting qui est fait. Noter que ce broadcasting est assez conservateur, on essaie de faire toutes les opérations possibles

The operation `A[:,np.newaxis]` creates a new dimension.

In [46]:
v = np.array([3,4,5])
print(v)
print(v[:,np.newaxis])
print(v[np.newaxis,:])

[3 4 5]
[[3]
 [4]
 [5]]
[[3 4 5]]


# Arrays of larger dimensions

In [47]:
a_3d_array = np.array([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])
a_3d_array

array([[[1, 2],
        [3, 4]],

       [[5, 6],
        [7, 8]]])

In [48]:
a_3d_array.size

8

Standard functions can also support arrays with more than 2 dimensions.

In [ ]:
a_multiarray = np.zeros((2,3,3,3))
print(a_multiarray, a_multiarray.shape)

# Searching for a maximum

### Maximum between 2 arrays

To compare two arrays (say $x$ and $y$) component by component, it is convenient to use `np.maximum`. It returns an array $z$ such that $ \forall i: z[i] = \max(x[i],y[i])$. 

In [49]:
np.max(np.array([2,3,4]))

4

In [53]:
np.maximum(A,B),A,B

(array([[10, 10],
        [20, 20],
        [30, 30]]),
 array([[10],
        [20],
        [30]]),
 array([1, 2]))

In [ ]:
np.maximum(np.array([2, 3, 4]), np.array([1, 5, 2]))

You can even broadcast.

In [ ]:
np.maximum(np.eye(2), [0.5, 2]) # broadcasting

### Highest component within an array

`np.max` and `np.argmax` respectively find the maximum entry of a given array along a specified axis, and its index. `np.min` and `np.argmin` perform similar functions.

In [54]:
A = np.array([[0, 1,3], [0, 5,7]])
A

array([[0, 1, 3],
       [0, 5, 7]])

In [56]:
A.shape

(2, 3)

## Attention : quand on écrit axis c'est pour dire qu'on compare les éléments le long de l'axis, pas qu'on donne le max pour chacun des lignes/colonnes le long axis

In [57]:
A.max(axis=0)

array([0, 5, 7])

In [58]:
A.argmax(axis=0)

array([0, 1, 1], dtype=int64)

In [59]:
A.max(axis=1)

array([3, 7])

In [60]:
A.min(axis=0)

array([0, 1, 3])

In [61]:
A.argmin(axis=0)

array([0, 0, 0], dtype=int64)

If `axis` is not specified, the maximum will be taken over all the entries of the matrix.

In [62]:
np.max(A) 

7

Note: if your array contains a nan, you can use `np.nanmax` in order to ignore those values while searching for the highest component.

## Summing all elements of an array

In a similar fashion as above, `np.sum` sums the elements of an array over a given axis.

In [63]:
A.sum( axis=0)

array([ 0,  6, 10])

In [64]:
A.sum(axis=1)

array([ 4, 12])

If `axis` is not specified, the sum is done over all the entries of the matrix.


In [65]:
A.sum()

16

# Sparse matrices in Scipy

Sparse matrices are available in the `sparse` module of the `scipy` library. 

In [66]:
import scipy.sparse as spr

In [67]:
n = 1000

print('size of sparse identity matrix of size '+str(n) +' in MB = ' + str(spr.identity(n).data.size  / (1024**2)))

print('size of dense identity matrix of size '+str(n) +' in MB  = ' + str(spr.identity(n).todense().nbytes  / (1024**2)))

size of sparse identity matrix of size 1000 in MB = 0.00095367431640625
size of dense identity matrix of size 1000 in MB  = 7.62939453125


Working with sparse matrices requires less storage. It is explained by the fact that while a dense matrix needs to encode every coefficient on a byte, sparse matrices only store the non-null coefficients. It is really convenient to work with such objects when it comes to matrices with really high sizes.

In [68]:
spr.identity(1000).data.size  , spr.identity(1000).todense().nbytes 

(1000, 8000000)

## Creating sparse matrices...

### ... with standard forms

In [69]:
I5 = spr.identity(5)
I5

<5x5 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements (1 diagonals) in DIAgonal format>

You can convert your sparse matrix into a dense one in order to visualise it. 

In [70]:
I5.todense()

matrix([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]])

### ... from a dense matrix

Let's create a dense matrix and make it sparse.

In [71]:
# import uniform module to create random numbers
from scipy.stats import uniform

In [72]:
np.random.seed(seed=42)
dense_matrix = uniform.rvs(size=16, loc = 0, scale=2) #List of 16 random draws between 0 and 2
dense_matrix = np.reshape(dense_matrix, (4, 4))
dense_matrix

array([[0.74908024, 1.90142861, 1.46398788, 1.19731697],
       [0.31203728, 0.31198904, 0.11616722, 1.73235229],
       [1.20223002, 1.41614516, 0.04116899, 1.9398197 ],
       [1.66488528, 0.42467822, 0.36364993, 0.36680902]])

In [73]:
dense_matrix[dense_matrix < 1] = 0 #Arbitrar criterion
dense_matrix

array([[0.        , 1.90142861, 1.46398788, 1.19731697],
       [0.        , 0.        , 0.        , 1.73235229],
       [1.20223002, 1.41614516, 0.        , 1.9398197 ],
       [1.66488528, 0.        , 0.        , 0.        ]])

In [74]:
sparse_matrix = spr.csr_matrix(dense_matrix)
print(sparse_matrix) #It prints a tuple giving the row and columns of the non-null component and its value.

  (0, 1)	1.9014286128198323
  (0, 2)	1.4639878836228102
  (0, 3)	1.1973169683940732
  (1, 3)	1.7323522915498704
  (2, 0)	1.2022300234864176
  (2, 1)	1.416145155592091
  (2, 3)	1.9398197043239886
  (3, 0)	1.6648852816008435


### ... from scratch

You can create two arrays containing respectively the rows and the column of the non-null coefficients.
A third array would give the value of the non-null coefficient. The result is as follows:

In [109]:
# row indices
row_ind = np.array([0, 1, 1, 3, 4])
# column indices
col_ind = np.array([0, 2, 4, 3, 4])
# coefficients
data = np.array([1, 2, 3, 4, 5], dtype=float)

mat_coo = spr.coo_matrix((data, (row_ind, col_ind)))
print(mat_coo)

  (0, 0)	1.0
  (1, 2)	2.0
  (1, 4)	3.0
  (3, 3)	4.0
  (4, 4)	5.0


## Essayons les deux méthodes

In [76]:
M = np.eye(1000)
M[15,20] = 10

In [81]:
print(spr.csr_matrix(M))

  (0, 0)	1.0
  (1, 1)	1.0
  (2, 2)	1.0
  (3, 3)	1.0
  (4, 4)	1.0
  (5, 5)	1.0
  (6, 6)	1.0
  (7, 7)	1.0
  (8, 8)	1.0
  (9, 9)	1.0
  (10, 10)	1.0
  (11, 11)	1.0
  (12, 12)	1.0
  (13, 13)	1.0
  (14, 14)	1.0
  (15, 15)	1.0
  (15, 20)	10.0
  (16, 16)	1.0
  (17, 17)	1.0
  (18, 18)	1.0
  (19, 19)	1.0
  (20, 20)	1.0
  (21, 21)	1.0
  (22, 22)	1.0
  (23, 23)	1.0
  :	:
  (975, 975)	1.0
  (976, 976)	1.0
  (977, 977)	1.0
  (978, 978)	1.0
  (979, 979)	1.0
  (980, 980)	1.0
  (981, 981)	1.0
  (982, 982)	1.0
  (983, 983)	1.0
  (984, 984)	1.0
  (985, 985)	1.0
  (986, 986)	1.0
  (987, 987)	1.0
  (988, 988)	1.0
  (989, 989)	1.0
  (990, 990)	1.0
  (991, 991)	1.0
  (992, 992)	1.0
  (993, 993)	1.0
  (994, 994)	1.0
  (995, 995)	1.0
  (996, 996)	1.0
  (997, 997)	1.0
  (998, 998)	1.0
  (999, 999)	1.0


In [110]:
row_ind = np.array(range(1000))
col_ind = np.array(range(1000))
row_ind = np.r_[row_ind,15]
col_ind = np.r_[col_ind,20]
data = np.r_[np.ones((1000,)),10]
Mat = spr.coo_matrix((data,(row_ind,col_ind)))

In [111]:
print(Mat)

  (0, 0)	1.0
  (1, 1)	1.0
  (2, 2)	1.0
  (3, 3)	1.0
  (4, 4)	1.0
  (5, 5)	1.0
  (6, 6)	1.0
  (7, 7)	1.0
  (8, 8)	1.0
  (9, 9)	1.0
  (10, 10)	1.0
  (11, 11)	1.0
  (12, 12)	1.0
  (13, 13)	1.0
  (14, 14)	1.0
  (15, 15)	1.0
  (16, 16)	1.0
  (17, 17)	1.0
  (18, 18)	1.0
  (19, 19)	1.0
  (20, 20)	1.0
  (21, 21)	1.0
  (22, 22)	1.0
  (23, 23)	1.0
  (24, 24)	1.0
  :	:
  (976, 976)	1.0
  (977, 977)	1.0
  (978, 978)	1.0
  (979, 979)	1.0
  (980, 980)	1.0
  (981, 981)	1.0
  (982, 982)	1.0
  (983, 983)	1.0
  (984, 984)	1.0
  (985, 985)	1.0
  (986, 986)	1.0
  (987, 987)	1.0
  (988, 988)	1.0
  (989, 989)	1.0
  (990, 990)	1.0
  (991, 991)	1.0
  (992, 992)	1.0
  (993, 993)	1.0
  (994, 994)	1.0
  (995, 995)	1.0
  (996, 996)	1.0
  (997, 997)	1.0
  (998, 998)	1.0
  (999, 999)	1.0
  (15, 20)	10.0


In [104]:
from scipy.sparse import issparse

In [108]:
issparse(Mat)

True

### Every common operation seen below works with sparse matrices.

## Attention : Mais ce ne sont pas forcément des matrices sparse qui sont renvoyées, va et vient entre les deux formats

In [113]:
I5 = spr.identity(5)
I5 + np.ones((5,5))

matrix([[2., 1., 1., 1., 1.],
        [1., 2., 1., 1., 1.],
        [1., 1., 2., 1., 1.],
        [1., 1., 1., 2., 1.],
        [1., 1., 1., 1., 2.]])

In [114]:
I5 + np.diag([1.,2.,3.,4.,5.])

matrix([[2., 0., 0., 0., 0.],
        [0., 3., 0., 0., 0.],
        [0., 0., 4., 0., 0.],
        [0., 0., 0., 5., 0.],
        [0., 0., 0., 0., 6.]])

In [115]:
I5 @ np.diag([1.,2.,3.,4.,5.])

array([[1., 0., 0., 0., 0.],
       [0., 2., 0., 0., 0.],
       [0., 0., 3., 0., 0.],
       [0., 0., 0., 4., 0.],
       [0., 0., 0., 0., 5.]])

In [116]:
kron_product = spr.kron(I5 , 10 *np.array([[1,2],[3,4]]))

In [117]:
kron_product.todense()

matrix([[10., 20.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [30., 40.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0., 10., 20.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0., 30., 40.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0., 10., 20.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0., 30., 40.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0., 10., 20.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0., 30., 40.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 10., 20.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 30., 40.]])

## Time comparison

In [130]:
import time
A = np.ones((1000,1000))
I5 = 3*spr.identity(1000)
B = I5.todense()

t0 = time.time()
A@B
t1 = time.time()
A@I5
t2 = time.time()

print((t1-t0)-(t2-t1))

0.053220272064208984


## No comprendo : pourquoi quand je change A aussi en sparse ça ne change pas vraiment le temps de réalisation ?

In [131]:
import time
A = np.ones((1000,1000))
I5 = 3*spr.identity(1000)
Ab = spr.csc_matrix(A)
B = I5.todense()

t0 = time.time()
A@B
t1 = time.time()
Ab@I5
t2 = time.time()

print((t1-t0)-(t2-t1))

0.03963065147399902


In [134]:
issparse(Ab)

True